### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import Audio
import torch.optim as optim
from types import SimpleNamespace
import scipy.signal as sc
import time
from sklearn.metrics import f1_score

from trainDataset import TrainDataset
from testDataset import TestDataset
#from trainDatasetNew import TrainDatasetNew
#from testDatasetNew import TestDatasetNew
from validation_split import get_dataloaders
from math_utils import logMagStft, ffts
from SpectrogramCNN import SpectrogramCNN
from train_utils import train, test
from evaluation_utils import get_mean_F1

### Parameters

In [2]:
validation_split = .2
do_plots = False
args = SimpleNamespace(batch_size=64, test_batch_size=64, epochs=3,
                       lr=0.01, momentum=0.5, seed=1, log_interval=200, 
                      net = SpectrogramCNN)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path_test =  './../data/test/kaggle-test/'
if not torch.cuda.is_available(): # adapt those paths on other machine
    print('no cuda')
    path_train = './../data/train-small/'
else:
    print('with cuda')
    path_train = './../data/kaggle-train/'
    
path_model = 'models/model.pt'
path_submission = 'submissions/'
    
sample_rate = 16000
nmbr_classes = 10

with cuda


### Original Dataset

In [3]:
# todo add in the classes the features and the fft data

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset(path_train, transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset(path_test, transform=toFloat)
print(len(testDataset))

269776
0


In [4]:
input_size = len(trainDataset[0][0])
print('input size: ',input_size)

input size:  64000


### Look at Original Data

In [5]:
if do_plots:
    # how many instruments are there?
    dummy_count = np.zeros(20)

    for sample in trainDataset:
        dummy_count[sample[1]] += 1

    labels_count = []
    for elem in dummy_count:
        if elem != 0:
            labels_count.append(elem)

    print(labels_count)

In [6]:
if do_plots:
    nmbr_classes = len(labels_count)
    print('nmbr_classes: ', nmbr_classes)

In [7]:
if do_plots:
    plt.plot(labels_count, '*')

In [8]:
if do_plots:
    # plot one of each

    done = np.zeros(nmbr_classes)
    examples = []

    for sample in trainDataset:
        if done[sample[1]] == 0:
            examples.append(sample)
            done[sample[1]] = 1

In [9]:
if do_plots:
    plt.subplot(431)
    plt.plot(examples[0][0])

    plt.subplot(432)
    plt.plot(examples[1][0])

    plt.subplot(433)
    plt.plot(examples[2][0])

    plt.subplot(434)
    plt.plot(examples[3][0])

    plt.subplot(435)
    plt.plot(examples[4][0])

    plt.subplot(436)
    plt.plot(examples[5][0])

    plt.subplot(437)
    plt.plot(examples[6][0])

    plt.subplot(438)
    plt.plot(examples[7][0])

    plt.subplot(439)
    plt.plot(examples[8][0])

    plt.subplot(4,3,10)
    plt.plot(examples[9][0])

    plt.show()

In [10]:
if do_plots:

    # plot one of each in FFT

    plt.subplot(431)
    plt.plot(ffts(examples[0][0]))

    plt.subplot(432)
    plt.plot(ffts(examples[1][0]))

    plt.subplot(433)
    plt.plot(ffts(examples[2][0]))

    plt.subplot(434)
    plt.plot(ffts(examples[3][0]))

    plt.subplot(435)
    plt.plot(ffts(examples[4][0]))

    plt.subplot(436)
    plt.plot(ffts(examples[5][0]))

    plt.subplot(437)
    plt.plot(ffts(examples[6][0]))

    plt.subplot(438)
    plt.plot(ffts(examples[7][0]))

    plt.subplot(439)
    plt.plot(ffts(examples[8][0]))

    plt.subplot(4,3,10)
    plt.plot(ffts(examples[9][0]))

    plt.show()

In [11]:
if do_plots:

    for sample in examples:
        display(Audio(sample[0], rate=sample_rate))

### Dataloaders

In [12]:
# validation split is done here

train_loader, validation_loader = get_dataloaders(trainDataset, 
                                                  batch_size = args.batch_size, 
                                                  validation_split = validation_split, 
                                                  shuffle_dataset = True, 
                                                  random_seed = None)

for samples, instrument_family_target in train_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break
        
for samples, instrument_family_target in validation_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break

torch.Size([64, 64000]) torch.Size([64]) tensor([0, 0, 0, 9, 6, 2, 6, 4, 4, 6, 4, 4, 7, 5, 0, 3, 8, 2, 4, 3, 4, 9, 3, 0,
        3, 5, 7, 7, 6, 0, 6, 3, 3, 4, 4, 8, 9, 0, 8, 4, 0, 0, 6, 3, 9, 6, 4, 3,
        0, 7, 0, 7, 0, 9, 4, 6, 4, 7, 6, 4, 5, 1, 5, 4])
tensor(-0.9651, dtype=torch.float64) tensor(0.9751, dtype=torch.float64)
['bass' 'bass' 'bass' 'vocal' 'organ' 'flute' 'organ' 'keyboard'
 'keyboard' 'organ' 'keyboard' 'keyboard' 'reed' 'mallet' 'bass' 'guitar'
 'string' 'flute' 'keyboard' 'guitar' 'keyboard' 'vocal' 'guitar' 'bass'
 'guitar' 'mallet' 'reed' 'reed' 'organ' 'bass' 'organ' 'guitar' 'guitar'
 'keyboard' 'keyboard' 'string' 'vocal' 'bass' 'string' 'keyboard' 'bass'
 'bass' 'organ' 'guitar' 'vocal' 'organ' 'keyboard' 'guitar' 'bass' 'reed'
 'bass' 'reed' 'bass' 'vocal' 'keyboard' 'organ' 'keyboard' 'reed' 'organ'
 'keyboard' 'mallet' 'brass' 'mallet' 'keyboard']
torch.Size([64, 64000]) torch.Size([64]) tensor([0, 3, 6, 3, 3, 4, 2, 9, 0, 4, 8, 7, 5, 0, 3, 5, 4, 6, 4, 4, 

In [13]:
test_loader = data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False) #!!! shuffle should be false
for samples in test_loader:
        print(samples.shape)
        print(torch.min(samples), torch.max(samples))
        break

### Main

In [ ]:
# Main
model = args.net(device).to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr, 
                      momentum=args.momentum)

info = {'lowest F1' : 100,
        'saved epoch' : None}

In [14]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch, start_time = time.time())
    f1 = get_mean_F1(model, validation_loader)
    print('after epoch {} got f1 score of {}'.format(epoch , f1))
    if f1 < info['lowest F1']:
        info['lowest F1'] = f1
        info['saved epoch'] = epoch 
        test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
        torch.save(model, path_model)
        print('currently best model --> saved')

Train Epoch: 1 [0/269776 (0%)]	Loss: 2.262139	F1: 0.1250	Runtime: 0.4
Train Epoch: 1 [12800/269776 (6%)]	Loss: 2.071094	F1: 0.2812	Runtime: 36.4
Train Epoch: 1 [25600/269776 (12%)]	Loss: 1.956254	F1: 0.2969	Runtime: 73.3
Train Epoch: 1 [38400/269776 (18%)]	Loss: 1.533787	F1: 0.3594	Runtime: 111.7
Train Epoch: 1 [51200/269776 (24%)]	Loss: 1.517673	F1: 0.3906	Runtime: 150.2
Train Epoch: 1 [64000/269776 (30%)]	Loss: 1.401508	F1: 0.4375	Runtime: 188.9
Train Epoch: 1 [76800/269776 (36%)]	Loss: 1.288415	F1: 0.5312	Runtime: 227.5
Train Epoch: 1 [89600/269776 (42%)]	Loss: 1.187410	F1: 0.6094	Runtime: 269.1
Train Epoch: 1 [102400/269776 (47%)]	Loss: 1.289615	F1: 0.6094	Runtime: 307.5
Train Epoch: 1 [115200/269776 (53%)]	Loss: 0.857002	F1: 0.6406	Runtime: 346.4
Train Epoch: 1 [128000/269776 (59%)]	Loss: 1.072261	F1: 0.6719	Runtime: 386.8
Train Epoch: 1 [140800/269776 (65%)]	Loss: 0.910911	F1: 0.6406	Runtime: 425.5
Train Epoch: 1 [153600/269776 (71%)]	Loss: 0.721228	F1: 0.8125	Runtime: 464.3
Trai